# How do socioeconomic factors affect crime rates in Seattle?
## Chantria Im, Ellie Velez, and Nawaf Osman

In [1]:
%%capture
%%script echo skipping # Skip this code cell

# Install packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests

In [2]:
# Import libraries
import pandas as pd
import requests
import re
import time

In [3]:
# Load datasets
crime_df = pd.read_csv('./data/SPD_Crime_Data__2008-Present_20240226.csv')
poverty_df = pd.read_csv('./data/Poverty_and_Near_Poverty_Map_Full_Data_data.csv', 
                         converters={'Census Tract': str.strip})
income_df = pd.read_csv('./data/ACSST5Y2018.S1903-Data.csv')

In [4]:
# Remove rows with missing Longitude and Latitude values
crime_df = crime_df[(crime_df['Longitude'] != 0) & (crime_df['Latitude'] != 0)]

# Filter for only 2018 entries
crime_df['Report DateTime'] = pd.to_datetime(crime_df['Report DateTime'], format='%m/%d/%Y %I:%M:%S %p')
crime_df = crime_df[crime_df['Report DateTime'].dt.year == 2018]

# Add Census Tract column
crime_df['Census Tract'] = pd.Series(dtype='string')

In [5]:
%%capture
%%script echo skipping # Skip this code cell

# Fill Census Tract column with corresponding census tract numbers
i = 0
while i < len(crime_df):
    # Make request to Census Geocoder API to get row's census tract data based on row's longtitude and latitude values
    payload = {'benchmark': 'Public_AR_Current', 'vintage': 'ACS2018_Current', 'x': crime_df['Longitude'].iloc[i], 
               'y': crime_df['Latitude'].iloc[i], 'format': 'json', 'layers': 'Census Tracts'}
    try:
        r = requests.get(f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates', params=payload)
    except requests.Timeout:
        time.sleep(300)
        continue
    # Extract the census tract number from the response and store it in the Census Tract column
    crime_df['Census Tract'].iloc[i] = re.findall('\d+\.\d+|\d+', 
                                                  r.json()['result']['geographies']['Census Tracts'][0]['NAME'])[0]
    i += 1

# Backup dataframe to disk 
crime_df.to_csv('./data/crime.csv')

In [6]:
pd.set_option('display.max_colwidth', None)

# Cache the dataframe
# %store crime_df
# Reload the dataframe from cache rather than recomputing (5+ hours)
%store -r crime_df

In [7]:
# Drop empty columns
poverty_df = poverty_df.drop(['% of Population Under 100% Poverty Line', '% of Population Under 200% Poverty Line', 
                              'Share Below Selected % of Poverty Level'], axis=1)

# Group data by census tract and calculate total poverty population for levels
poverty_by_tract = poverty_df[poverty_df['Name'] != 'Total'].groupby('Census Tract').agg(
    {'Population Under 100% Poverty Level': 'sum', 'Population Under 200% Poverty Level': 'sum'})

# For each census tract fill designated 'Total' row with total poverty populations for levels
poverty_df.loc[
    poverty_df['Name'] == 'Total', 
    'Population Under 100% Poverty Level'] = poverty_df.loc[
    poverty_df['Name'] == 'Total', 'Census Tract'].map(poverty_by_tract['Population Under 100% Poverty Level']).values
poverty_df.loc[
    poverty_df['Name'] == 'Total', 
    'Population Under 200% Poverty Level'] = poverty_df.loc[
    poverty_df['Name'] == 'Total', 'Census Tract'].map(poverty_by_tract['Population Under 200% Poverty Level']).values

# Filter for only designated 'Total' rows
poverty_df = poverty_df[poverty_df['Name'] == 'Total']

In [8]:
# Remove rows with missing median income values
income_df = income_df[pd.to_numeric(income_df['S1903_C03_001E'], errors='coerce').notna()] # 53.02 University District Missing

# Extract and set census tract numbers
for i in range(len(income_df)):
    income_df.iat[i, income_df.columns.get_loc('NAME')] = re.findall('\d+\.\d+|\d+', income_df['NAME'].iloc[i])[0]
    
# Edit column names
income_df = income_df.rename(columns={'NAME': 'Census Tract', 'S1903_C03_001E': 'Median Income'})

# Set median income column to numeric type
income_df['Median Income'] = pd.to_numeric(income_df['Median Income'])

In [9]:
# Drop un-needed columns
crime_df = crime_df.filter(['Census Tract', 'Offense Start DateTime', 'Offense End DateTime', 'Report DateTime', 
                            'Offense Parent Group'])
poverty_df = poverty_df.filter(['Census Tract', 'Population Under 100% Poverty Level', 
                                'Population Under 200% Poverty Level', 'Total Population'])
income_df = income_df.filter(['Census Tract', 'Median Income'])

In [10]:
# Merge datasets
merged_df = crime_df.merge(poverty_df, on='Census Tract')
merged_df = merged_df.merge(income_df, on='Census Tract')
merged_df

,Census Tract,Offense Start DateTime,Offense End DateTime,Report DateTime,Offense Parent Group,Population Under 100% Poverty Level,Population Under 200% Poverty Level,Total Population,Median Income
0,56,12/31/2018 11:39:00 PM,NaN,2018-12-31 23:39:00,STOLEN PROPERTY OFFENSES,170,353,6954,170031
1,56,12/29/2018 09:05:00 AM,NaN,2018-12-29 09:05:00,EXTORTION/BLACKMAIL,170,353,6954,170031
2,56,12/24/2018 06:00:00 PM,12/25/2018 11:00:00 AM,2018-12-25 14:14:00,LARCENY-THEFT,170,353,6954,170031
3,56,12/24/2018 09:30:00 AM,NaN,2018-12-24 10:21:00,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,170,353,6954,170031
4,56,12/18/2018 04:00:00 PM,12/20/2018 06:00:00 PM,2018-12-23 18:20:00,LARCENY-THEFT,170,353,6954,170031
...,...,...,...,...,...,...,...,...,...
72432,213,04/16/2018 02:42:00 PM,NaN,2018-04-16 16:04:00,ARSON,294,595,3941,80227
72433,213,03/19/2018 09:20:00 PM,NaN,2018-03-19 21:20:00,ASSAULT OFFENSES,294,595,3941,80227
72434,213,02/01/2018 12:28:00 AM,NaN,2018-02-01 00:28:00,ROBBERY,294,595,3941,80227
72435,213,01/27/2018 07:49:00 PM,NaN,2018-01-27 19:49:00,LARCENY-THEFT,294,595,3941,80227


In [11]:
# Group merged dataset by census tract
grouped_df = merged_df.groupby('Census Tract').agg({
    'Offense Parent Group': 'count',
    'Population Under 100% Poverty Level': 'first',
    'Population Under 200% Poverty Level': 'first',
    'Total Population': 'first',
    'Median Income': 'first'
})
grouped_df = grouped_df.rename(columns={'Offense Parent Group': 'Total Offenses'})

# Calculate crime rate per census tract
# A crime rate is calculated by dividing the number of reported crimes by the total population.
# The result is then multiplied by 100,000.
grouped_df['Crime Rate per 100,000'] = (grouped_df['Total Offenses'] / grouped_df['Total Population']) * 100000
grouped_df['Crime Rate per 100,000'] = grouped_df['Crime Rate per 100,000'].round(2)
grouped_df

# Calculate crime rate per census tract by offense type
offense_type_df = merged_df.groupby(['Census Tract', 'Offense Parent Group', 'Population Under 100% Poverty Level', 'Population Under 200% Poverty Level',
                                     'Total Population', 'Median Income']).size().reset_index(name='Offense Count')
offense_type_df['Crime Rate per 100,000 for Offense Type'] = (offense_type_df['Offense Count'] / offense_type_df['Total Population']) * 100000
offense_type_df['Crime Rate per 100,000 for Offense Type'] = offense_type_df['Crime Rate per 100,000 for Offense Type'].round(2)

# Reorder columns
col_order = ['Census Tract', 'Offense Parent Group', 'Offense Count', 'Population Under 100% Poverty Level', 'Population Under 200% Poverty Level',
             'Total Population', 'Median Income', 'Crime Rate per 100,000 for Offense Type']
offense_type_df= offense_type_df[col_order]
offense_type_df